In [159]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR

% matplotlib inline

UsageError: Line magic function `%` not found.


In [160]:
def remove_bad_players(frame):


    return frame[((frame['TRB'] > 5) | (frame['AST'] > 5)) & (frame['PTS'] > 15) & (frame['Win%'] > .5)]



In [161]:
df = pd.read_csv('../Data/Players With Award Data/complete_players_with_award_voting.csv', index_col=0)

df = df.drop(columns=['STAR', 'DPOY_Rank', 'DPOY_First', 'DPOY_Pts Won', 'DPOY_Pts Max', 'DPOY_Share',
                      'DPOY', 'MIP_Rank', 'MIP_First', 'MIP_Pts Won', 'MIP_Pts Max',
                      'MIP_Share', 'MIP', 'MVP_First', 'MVP_Pts Won',
                      'MVP_Pts Max', 'ROTY_Rank', 'ROTY_First',
                      'ROTY_Pts Won', 'ROTY_Pts Max', 'ROTY_Share', 'ROTY', 'SMOTY_Rank',
                      'SMOTY_First', 'SMOTY_Pts Won', 'SMOTY_Pts Max', 'SMOTY_Share',
                      'SMOTY'])

df = remove_bad_players(df)

In [169]:
train_set = df.copy()
test_set = pd.read_csv('/Users/chaseallbright/Downloads/2022.csv', index_col=0)
test_set = remove_bad_players(test_set)
train_set.columns

x_train = df.drop(columns=['MVP_Rank', 'MVP_Share', 'MVP'])
y_train = train_set['MVP_Share']



KeyError: 'Win%'

In [170]:
cm = train_set.corr().MVP_Share.apply(np.absolute).sort_values(ascending=False)

In [171]:
# list(cm[cm > .1].to_dict().keys())

In [172]:
feature_list = ['MP', 'PTS', 'FG', 'FGA', 'TOV', 'AST', 'STL', 'TRB', 'BLK']

x_train = train_set[feature_list]
x_test = test_set[feature_list]

In [173]:
num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
    ('linear_reg', LinearRegression())
])
ppln_fitted = num_pipeline.fit(x_train, y_train, )

In [174]:
y_pred = ppln_fitted.predict(x_test)

test_set['pred'] = y_pred
test_set['pred_scaled'] = MinMaxScaler().fit_transform(y_pred.reshape(-1, 1))
test_set['pred_rank'] = test_set.pred.rank(ascending=False).astype("int")

In [175]:
test_set.sort_values('pred_rank', ascending=True).head(10)[
    ['Player', 'G', 'PTS', 'FG%', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'pred']]

,Player,G,PTS,FG%,AST,TRB,STL,BLK,TOV,pred
13,Giannis Antetokounmpo,33.0,28.5,0.540,5.9,11.4,1.1,1.5,3.3,0.433912
313,Nikola Jokić,32.0,26.0,0.567,7.0,14.1,1.4,0.8,3.7,0.379374
168,Joel Embiid,27.0,26.8,0.475,4.3,10.7,1.1,1.4,2.9,0.346356
161,Kevin Durant,32.0,29.9,0.519,5.8,7.8,0.7,0.9,3.2,0.334414
293,LeBron James,28.0,28.7,0.521,6.6,7.4,1.8,1.2,3.3,0.325950
653,Trae Young,34.0,28.3,0.458,9.6,4.1,0.9,0.1,4.1,0.291749
230,James Harden,33.0,22.3,0.414,9.7,8.1,1.3,0.7,4.8,0.225285
94,Jimmy Butler,23.0,23.2,0.500,5.4,5.9,2.0,0.4,2.0,0.223413
132,Stephen Curry,35.0,26.8,0.420,6.1,5.3,1.4,0.5,3.4,0.217647
140,DeMar DeRozan,33.0,26.5,0.489,4.6,5.0,0.9,0.3,2.2,0.205387
